# Práctica 1: Reglas de asociación

Imports necesarios para la realización de la práctica

In [ ]:
import numpy as np
import pandas as pd
import sklearn as sk
import mlxtend as mlx
import apyori as apy
import matplotlib as mpl